In [6]:
import os
import cv2
import xml.etree.ElementTree as ET
import shutil
import random
import matplotlib.pyplot as plt

In [2]:
base_dir = 'images'
jpeg_dir = 'jpeg'
xml_dir = 'xml'

os.makedirs(jpeg_dir, exist_ok=True)
os.makedirs(xml_dir, exist_ok=True)

for filename in os.listdir(base_dir):
    if filename.endswith('.jpeg'):
        shutil.copy(os.path.join(base_dir, filename), os.path.join(jpeg_dir, filename))
    elif filename.endswith('.xml'):
        shutil.copy(os.path.join(base_dir, filename), os.path.join(xml_dir, filename))

print('Files copied separately in their respective directory.')

Files copied separately in their respective directory.


In [3]:
jpeg_count = len([filename for filename in os.listdir(jpeg_dir) if filename.endswith('.jpeg')])
xml_count = len([filename for filename in os.listdir(xml_dir) if filename.endswith('.xml')])

print(f"Jpeg count: {jpeg_count}")
print(f"xml count: {xml_count}")


Jpeg count: 228
xml count: 225


In [7]:
jpeg_dir = "jpeg"
xml_dir = "xml"
dataset_dir = "dataset"

train_images_dir = os.path.join(dataset_dir, 'images', 'train')
train_labels_dir = os.path.join(dataset_dir, 'labels', 'train')
val_images_dir = os.path.join(dataset_dir, 'images', 'val')
val_labels_dir = os.path.join(dataset_dir, 'labels', 'val')

os.makedirs(train_images_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(val_images_dir, exist_ok=True)
os.makedirs(val_labels_dir, exist_ok=True)

def convert_voc_to_yolo(xml_file, img_width, img_height):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    yolo_data = []

    for obj in root.findall('object'):
        class_name = obj.find('name').text
        class_id = 0

        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)

        # Convert to YOLO format
        x_center = (xmin + xmax) / 2 / img_width
        y_center = (ymin + ymax) / 2 / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height

        yolo_data.append(f"{class_id} {x_center} {y_center} {width} {height}")

    return yolo_data

image_files = []

for filename in os.listdir(jpeg_dir):
    if filename.endswith('.jpeg'):
        image_files.append(filename)

random.shuffle(image_files)

split_index = int(len(image_files) * 0.8)
train_files = image_files[:split_index]
val_files = image_files[split_index:]

for filename in train_files + val_files:
    image_path = os.path.join(jpeg_dir, filename)

    if filename in train_files:
        target_image_dir = train_images_dir
        target_label_dir = train_labels_dir
    else:
        target_image_dir = val_images_dir
        target_label_dir = val_labels_dir

    shutil.copy(image_path, os.path.join(target_image_dir, filename))

    xml_file_path = os.path.join(xml_dir, os.path.splitext(filename)[0] + '.xml')

    if os.path.exists(xml_file_path):
        img = cv2.imread(image_path)
        img_height, img_width, _ = img.shape

        yolo_labels = convert_voc_to_yolo(xml_file_path, img_width, img_height)
        label_file_path = os.path.join(target_label_dir, f"{os.path.splitext(filename)[0]}.txt")

        with open(label_file_path, 'w') as f:
            for line in yolo_labels:
                f.write(line + '\n')
    
    else:
        print(f"No XML file found for {filename}, skipping label generation.")

print("Dataset structure created with train and validation sets.")


No XML file found for N182.jpeg, skipping label generation.
No XML file found for N249.jpeg, skipping label generation.
No XML file found for N180.jpeg, skipping label generation.
Dataset structure created with train and validation sets.


In [4]:
yaml_content = """train: C:/Users/2003s/Desktop/number-plate-detection/dataset/images/train
val: C:/Users/2003s/Desktop/number-plate-detection/dataset/images/train

nc: 1 # Number of classes
names: ['number_plate'] # List of class names
"""

with open('dataset/dataset.yaml', 'w') as file:
    file.write(yaml_content)

print("dataset.yaml file has been created.")

dataset.yaml file has been created.


In [5]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

model.train(data='dataset/dataset.yaml', epochs=100, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.86 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.1.10 🚀 Python-3.12.4 torch-2.2.0+cpu CPU (11th Gen Intel Core(TM) i5-11300H 3.10GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset/dataset.yaml, epochs=100, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fa

train: Scanning C:\Users\2003s\Desktop\number-plate-detection\dataset\labels\train... 181 images, 1 backgrounds, 0 corrupt: 100%|██████████| 182/182 [00:00<00:00, 341.11it/s]

train: New cache created: C:\Users\2003s\Desktop\number-plate-detection\dataset\labels\train.cache



val: Scanning C:\Users\2003s\Desktop\number-plate-detection\dataset\labels\train.cache... 181 images, 1 backgrounds, 0 corrupt: 100%|██████████| 182/182 [00:00<?, ?it/s]


Plotting labels to runs\detect\train2\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs\detect\train2
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        182        181          1          1      0.995      0.902



100 epochs completed in 2.435 hours.
Optimizer stripped from runs\detect\train2\weights\last.pt, 6.3MB
Optimizer stripped from runs\detect\train2\weights\best.pt, 6.3MB

Validating runs\detect\train2\weights\best.pt...
Ultralytics YOLOv8.1.10 🚀 Python-3.12.4 torch-2.2.0+cpu CPU (11th Gen Intel Core(TM) i5-11300H 3.10GHz)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 6/6 [00:15<00:00,  2.54s/it]


                   all        182        181          1          1      0.995      0.908
Speed: 1.4ms preprocess, 71.2ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs\detect\train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000029A514FB0B0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [13]:
import cv2
import matplotlib.pyplot as plt
model = YOLO('runs/detect/train2/weights/best.pt')

test_images_dir = 'TEST'
output_dir = 'test_results'
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(test_images_dir):
    if filename.endswith(('.jpeg', '.jpg', '.png')):
        image_path = os.path.join(test_images_dir, filename)

        results = model.predict(source=image_path, save=True, save_dir=output_dir)

        if results:
            img = cv2.imread(image_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            for result in results:
                boxes = result.boxes
                if len(boxes) == 0:
                    print(f"No detections for {filename}.")
                    continue

                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    confidence = box.conf[0].cpu().item()
                    class_id = int(box.cls[0].cpu().item())

                    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (255,0,0), 2)
                    label = f'Class {class_id} {confidence:.2f}'
                    cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

            plt.imshow(img)
            plt.title(f'Detections for {filename}')
            plt.axis('off')
            plt.show()

        else:
            print(f"No results for {filename}")

print("DONE!")


image 1/1 c:\Users\2003s\Desktop\number-plate-detection\TEST\TEST.jpeg: 544x640 1 number_plate, 104.0ms
Speed: 33.5ms preprocess, 104.0ms inference, 0.0ms postprocess per image at shape (1, 3, 544, 640)
Results saved to runs\detect\predict5


<Figure size 640x480 with 1 Axes>

DONE!


In [14]:
models_dir = 'models'

os.makedirs(models_dir, exist_ok=True)

model.save('runs/detect/train2/weights/best.pt')

print("Model weights have been saved.")

Model weights have been saved.
